# Day5_2: 실습 프로젝트 - 더러운 전자상거래 데이터 정제 (정답)

## 📚 프로젝트 목표

**Part 1: 데이터 품질 진단**
1. 결측치 탐색 및 처리 전략 수립하기
2. 이상치 탐지 및 비즈니스 로직 검증하기
3. 데이터 타입 오류 파악하기

**Part 2: 데이터 정제 실행**
1. 결측치 처리 (삭제/대체)
2. 이상치 처리 및 수정
3. 데이터 타입 변환 및 포맷 통일
4. 중복 레코드 제거
5. Plotly 대시보드로 전체 과정 시각화
6. 정제된 데이터 저장

---

## 📦 환경 설정

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

print("✅ 라이브러리 임포트 완료")

In [ ]:
# 더러운 데이터 로드
df_raw = pd.read_csv('datasets/dirty_ecommerce_data.csv')

# 원본 보존용 복사
df = df_raw.copy()

# 원본 레코드 수 저장 (나중에 비교용)
original_count = len(df)

print(f"📊 원본 데이터 로드 완료: {len(df)}건")
print(f"📋 컬럼 수: {len(df.columns)}개")
df.head(10)

---

# Part 1: 데이터 품질 진단

---

## Mission 1: 전체 데이터 구조 파악하기

In [ ]:
# 기본 정보 확인
df.info()

In [ ]:
# 데이터 타입 확인
print("===== 컬럼별 데이터 타입 =====")
print(df.dtypes)

### 💡 진단 결과

- order_date가 object(문자열)로 되어 있음 → datetime으로 변환 필요
- price가 object로 되어 있음 → 숫자로 변환 필요 (문자열 혼입 가능성)

---

## Mission 2: 결측치 탐색하기

In [ ]:
# 컬럼별 결측치 개수
missing_counts = df.isnull().sum()
print("===== 컬럼별 결측치 개수 =====")
print(missing_counts[missing_counts > 0])

In [ ]:
# 결측치 비율 계산
missing_ratio = (df.isnull().sum() / len(df) * 100).round(2)
print("\n===== 컬럼별 결측치 비율(%) =====")
print(missing_ratio[missing_ratio > 0])

In [ ]:
# Plotly 막대 차트로 결측치 시각화
missing_df = pd.DataFrame({
    'column': missing_counts[missing_counts > 0].index,
    'missing_count': missing_counts[missing_counts > 0].values,
    'missing_ratio': missing_ratio[missing_ratio > 0].values
})

fig = px.bar(
    missing_df,
    x='column',
    y='missing_count',
    title='컬럼별 결측치 현황',
    labels={'column': '컬럼명', 'missing_count': '결측치 개수'},
    text='missing_count',
    color='missing_ratio',
    color_continuous_scale='Reds'
)

fig.update_traces(texttemplate='%{text}건', textposition='outside')
fig.update_layout(height=400)
fig.show()

### 💡 진단 결과

- customer_grade, delivery_status, payment_method, region, price에 결측치 존재
- 각 컬럼별로 5-15% 정도의 결측치 비율
- 처리 전략: 비즈니스 로직에 맞게 최빈값 또는 평균값으로 대체

---

## Mission 3: 이상치 탐지하기

In [ ]:
# quantity 기술 통계
print("===== 수량(quantity) 기술 통계 =====")
print(df['quantity'].describe())

In [ ]:
# 음수 수량 레코드 찾기
negative_qty = df[df['quantity'] < 0]
print(f"\n⚠️ 음수 수량 레코드: {len(negative_qty)}건")
print(negative_qty[['order_id', 'product', 'quantity', 'delivery_status']].head())

In [ ]:
# price를 숫자로 변환 (타입 오류 무시)
df['price_numeric'] = pd.to_numeric(df['price'], errors='coerce')

print("===== 가격(price) 기술 통계 =====")
print(df['price_numeric'].describe())

In [ ]:
# 비정상 가격 찾기
too_high = df[df['price_numeric'] > 1000000]
too_low = df[df['price_numeric'] < 100]

print(f"\n⚠️ 100만원 초과 가격: {len(too_high)}건")
print(f"⚠️ 100원 미만 가격: {len(too_low)}건")

In [ ]:
# Plotly 박스플롯으로 가격 분포 시각화
fig = px.box(
    df,
    y='price_numeric',
    title='가격 분포 (이상치 포함)',
    labels={'price_numeric': '가격 (원)'},
    points='outliers'  # 이상치만 점으로 표시
)

fig.update_layout(height=500)
fig.show()

### 💡 진단 결과

- 수량 음수 10건: 반품으로 잘못 입력된 것으로 추정
- 가격이 100만원 초과 5건, 100원 미만 5건: 입력 오류로 판단
- 처리 방법: 음수는 절대값, 비정상 가격은 제품 평균가로 대체

---

## Mission 4: 데이터 타입 오류 찾기

In [ ]:
# order_date의 다양한 형식 확인
print("===== order_date 샘플 (20건) =====")
print(df['order_date'].sample(20).values)

In [ ]:
# category 고유값 확인 (공백 문제)
print("\n===== category 고유값 =====")
print(df['category'].unique())

In [ ]:
# customer_grade 대소문자 혼용 확인
print("\n===== customer_grade 분포 (대소문자 구분) =====")
print(df['customer_grade'].value_counts())

In [ ]:
# price에서 '원' 텍스트 포함된 레코드
price_with_won = df[df['price'].astype(str).str.contains('원', na=False)]
print(f"\n⚠️ 가격에 '원' 텍스트 포함: {len(price_with_won)}건")
print(price_with_won[['order_id', 'product', 'price']].head())

### 💡 진단 결과

- order_date: '2024-01-15', '2024/01/15', '20240115' 등 다양한 형식 혼재
- category: 공백 포함된 값 존재 (예: '전자제품 ', ' 의류')
- customer_grade: 'VIP', 'vip' 등 대소문자 혼용
- price: '50000원' 같은 문자열 5건

**처리 방법**: strip(), upper(), replace() 등으로 포맷 통일

---

## Mission 5: 중복 레코드 확인하기

In [ ]:
# 완전 중복 레코드 개수
duplicate_count = df.duplicated().sum()
print(f"===== 중복 레코드 개수 =====")
print(f"완전 중복: {duplicate_count}건")

In [ ]:
# order_id 중복 확인
order_id_dup = df['order_id'].duplicated().sum()
print(f"\norder_id 중복: {order_id_dup}건")

# 중복된 order_id 샘플
if order_id_dup > 0:
    dup_ids = df[df['order_id'].duplicated(keep=False)]['order_id'].unique()[:3]
    print(f"\n중복된 order_id 샘플: {dup_ids}")

### 💡 진단 결과

- 완전 중복 레코드 10건 발견
- 처리 방법: drop_duplicates()로 첫 번째만 유지

---

## Mission 6: 비즈니스 로직 검증하기

In [ ]:
# 취소/반품인데 금액이 있는 레코드
invalid_cancellation = df[
    (df['delivery_status'].isin(['취소', '반품'])) & 
    (df['total_amount'] > 0)
]

print(f"===== 비즈니스 로직 위배 =====")
print(f"취소/반품인데 금액 > 0: {len(invalid_cancellation)}건")
print(invalid_cancellation[['order_id', 'delivery_status', 'total_amount']].head())

### 💡 진단 결과

- 취소/반품 주문인데 total_amount가 남아있는 경우 5건
- 처리 방법: 취소/반품은 total_amount를 0으로 수정

---

# Part 2: 데이터 정제 실행

---

## Mission 7: 데이터 타입 정제하기

In [ ]:
# 1. order_date를 datetime으로 변환
# 슬래시를 하이픈으로 변경 후 변환
df['order_date'] = df['order_date'].str.replace('/', '-')
df['order_date'] = pd.to_datetime(df['order_date'], format='%Y-%m-%d', errors='coerce')

print("✅ order_date → datetime 변환 완료")
print(f"   타입: {df['order_date'].dtype}")

In [ ]:
# 2. category와 delivery_status 공백 제거
df['category'] = df['category'].str.strip()
df['delivery_status'] = df['delivery_status'].str.strip()

print("✅ category, delivery_status 공백 제거 완료")
print(f"   category 고유값: {df['category'].unique()}")

In [ ]:
# 3. customer_grade 대문자로 통일
df['customer_grade'] = df['customer_grade'].str.upper()

print("✅ customer_grade 대문자 통일 완료")
print(f"   고유값: {df['customer_grade'].unique()}")

In [ ]:
# 4. price에서 '원' 제거 후 숫자로 변환
df['price'] = df['price'].astype(str).str.replace('원', '')
df['price'] = pd.to_numeric(df['price'], errors='coerce')

print("✅ price → 숫자 변환 완료")
print(f"   타입: {df['price'].dtype}")

---

## Mission 8: 이상치 처리하기

In [ ]:
# 1. 음수 수량을 양수로 변환
negative_mask = df['quantity'] < 0
df.loc[negative_mask, 'quantity'] = df.loc[negative_mask, 'quantity'].abs()

print(f"✅ 음수 수량 → 양수 변환: {negative_mask.sum()}건")

In [ ]:
# 2. 비정상 가격을 제품별 평균가로 대체
# 먼저 제품별 정상 가격 평균 계산 (100원 ~ 100만원 사이)
normal_price_mask = (df['price'] >= 100) & (df['price'] <= 1000000)
product_avg_price = df[normal_price_mask].groupby('product')['price'].mean()

# 비정상 가격 찾기
abnormal_price_mask = (df['price'] < 100) | (df['price'] > 1000000)

# 제품별 평균가로 대체
for idx in df[abnormal_price_mask].index:
    product = df.loc[idx, 'product']
    if product in product_avg_price.index:
        df.loc[idx, 'price'] = product_avg_price[product]

print(f"✅ 비정상 가격 → 제품 평균가 대체: {abnormal_price_mask.sum()}건")

---

## Mission 9: 결측치 처리하기

In [ ]:
# 1. customer_grade 최빈값으로 대체
most_common_grade = df['customer_grade'].mode()[0]
df['customer_grade'].fillna(most_common_grade, inplace=True)

print(f"✅ customer_grade 결측치 → '{most_common_grade}' 대체")

In [ ]:
# 2. delivery_status → '준비중'으로 대체
df['delivery_status'].fillna('준비중', inplace=True)
print("✅ delivery_status 결측치 → '준비중' 대체")

In [ ]:
# 3. payment_method → '카드'로 대체
df['payment_method'].fillna('카드', inplace=True)
print("✅ payment_method 결측치 → '카드' 대체")

In [ ]:
# 4. region 최빈값으로 대체
most_common_region = df['region'].mode()[0]
df['region'].fillna(most_common_region, inplace=True)

print(f"✅ region 결측치 → '{most_common_region}' 대체")

In [ ]:
# 5. price를 제품별 평균가로 대체
df['price'] = df.groupby('product')['price'].transform(
    lambda x: x.fillna(x.mean())
)

print("✅ price 결측치 → 제품별 평균가 대체")

In [ ]:
# 결측치 처리 확인
print("\n===== 결측치 처리 후 확인 =====")
print(df.isnull().sum())

---

## Mission 10: 중복 레코드 제거하기

In [ ]:
# 중복 제거 전 레코드 수
before_dedup = len(df)

# 중복 제거 (첫 번째만 유지)
df.drop_duplicates(inplace=True)

# 인덱스 재설정
df.reset_index(drop=True, inplace=True)

after_dedup = len(df)
print(f"✅ 중복 레코드 제거 완료")
print(f"   제거 전: {before_dedup}건")
print(f"   제거 후: {after_dedup}건")
print(f"   제거된 레코드: {before_dedup - after_dedup}건")

---

## Mission 11: 비즈니스 로직 수정하기

In [ ]:
# 1. 취소/반품 주문의 total_amount를 0으로
cancellation_mask = df['delivery_status'].isin(['취소', '반품'])
df.loc[cancellation_mask, 'total_amount'] = 0

print(f"✅ 취소/반품 주문의 total_amount → 0 수정: {cancellation_mask.sum()}건")

In [ ]:
# 2. 모든 레코드의 total_amount 재계산
df['total_amount'] = df['quantity'] * df['price']

# 단, 취소/반품은 다시 0으로
df.loc[cancellation_mask, 'total_amount'] = 0

print("✅ total_amount 재계산 완료")

---

## Mission 12: Plotly 대시보드 생성하기

### 정제 전후 비교 대시보드

In [ ]:
# 2x2 서브플롯 생성
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '1. 레코드 수 변화',
        '2. 결측치 처리 현황',
        '3. 카테고리별 매출 분포',
        '4. 고객 등급별 주문 건수'
    ),
    specs=[
        [{'type': 'bar'}, {'type': 'bar'}],
        [{'type': 'bar'}, {'type': 'bar'}]
    ]
)

# 1. 레코드 수 변화
record_counts = pd.DataFrame({
    '구분': ['정제 전', '정제 후'],
    '레코드수': [original_count, len(df)]
})

fig.add_trace(
    go.Bar(
        x=record_counts['구분'],
        y=record_counts['레코드수'],
        text=record_counts['레코드수'],
        textposition='outside',
        marker_color=['#FF6B6B', '#4ECDC4']
    ),
    row=1, col=1
)

# 2. 결측치 처리 현황
missing_before = df_raw.isnull().sum().sum()
missing_after = df.isnull().sum().sum()

missing_comparison = pd.DataFrame({
    '구분': ['정제 전', '정제 후'],
    '결측치수': [missing_before, missing_after]
})

fig.add_trace(
    go.Bar(
        x=missing_comparison['구분'],
        y=missing_comparison['결측치수'],
        text=missing_comparison['결측치수'],
        textposition='outside',
        marker_color=['#FF6B6B', '#4ECDC4']
    ),
    row=1, col=2
)

# 3. 카테고리별 매출 분포
category_sales = df[df['delivery_status'] != '취소'].groupby('category')['total_amount'].sum().sort_values(ascending=False)

fig.add_trace(
    go.Bar(
        x=category_sales.index,
        y=category_sales.values,
        text=[f"{v/10000:.0f}만" for v in category_sales.values],
        textposition='outside',
        marker_color='#95E1D3'
    ),
    row=2, col=1
)

# 4. 고객 등급별 주문 건수
grade_counts = df['customer_grade'].value_counts()

fig.add_trace(
    go.Bar(
        x=grade_counts.index,
        y=grade_counts.values,
        text=grade_counts.values,
        textposition='outside',
        marker_color='#F38181'
    ),
    row=2, col=2
)

# 레이아웃 업데이트
fig.update_layout(
    title_text="📊 전자상거래 데이터 정제 대시보드",
    title_font_size=20,
    showlegend=False,
    height=800,
    template='plotly_white'
)

fig.update_xaxes(title_text="", row=1, col=1)
fig.update_xaxes(title_text="", row=1, col=2)
fig.update_xaxes(title_text="카테고리", row=2, col=1)
fig.update_xaxes(title_text="고객 등급", row=2, col=2)

fig.update_yaxes(title_text="레코드 수", row=1, col=1)
fig.update_yaxes(title_text="결측치 개수", row=1, col=2)
fig.update_yaxes(title_text="총 매출 (원)", row=2, col=1)
fig.update_yaxes(title_text="주문 건수", row=2, col=2)

fig.show()

# HTML로 저장
fig.write_html('datasets/data_cleaning_dashboard.html')
print("\n✅ 대시보드 HTML 저장 완료: datasets/data_cleaning_dashboard.html")

---

## Mission 13: 정제된 데이터 저장하기

In [ ]:
# 정제된 데이터 CSV로 저장
df.to_csv('datasets/clean_ecommerce_data.csv', index=False, encoding='utf-8-sig')

print("✅ 정제된 데이터 저장 완료: datasets/clean_ecommerce_data.csv")

In [ ]:
# 최종 품질 리포트
print("\n" + "="*50)
print("===== 데이터 정제 완료 리포트 =====")
print("="*50)
print(f"\n📊 레코드 수")
print(f"   원본: {original_count}건")
print(f"   정제 후: {len(df)}건")
print(f"   제거된 레코드: {original_count - len(df)}건 ({(original_count - len(df)) / original_count * 100:.1f}%)")

print(f"\n🔍 결측치 현황")
print(f"   정제 전: {missing_before}개")
print(f"   정제 후: {df.isnull().sum().sum()}개")

print(f"\n📈 데이터 품질 지표")
print(f"   완전성: {(1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100:.2f}%")
print(f"   중복 제거율: {(before_dedup - after_dedup) / before_dedup * 100:.1f}%")

print(f"\n💰 비즈니스 인사이트")
print(f"   총 매출: {df[df['delivery_status'] != '취소']['total_amount'].sum():,.0f}원")
print(f"   평균 주문 금액: {df[df['delivery_status'] != '취소']['total_amount'].mean():,.0f}원")
print(f"   VIP 고객 비율: {(df['customer_grade'] == 'VIP').sum() / len(df) * 100:.1f}%")

print(f"\n📁 결과 파일")
print(f"   정제된 데이터: datasets/clean_ecommerce_data.csv")
print(f"   대시보드: datasets/data_cleaning_dashboard.html")
print("\n" + "="*50)

---

## 📊 프로젝트 완료 체크리스트

### Part 1: 진단 단계
- [x] Mission 1: 데이터 구조 파악
- [x] Mission 2: 결측치 탐색 및 시각화
- [x] Mission 3: 이상치 탐지 및 박스플롯
- [x] Mission 4: 데이터 타입 오류 파악
- [x] Mission 5: 중복 레코드 확인
- [x] Mission 6: 비즈니스 로직 검증

### Part 2: 정제 단계
- [x] Mission 7: 데이터 타입 변환
- [x] Mission 8: 이상치 처리
- [x] Mission 9: 결측치 대체
- [x] Mission 10: 중복 제거
- [x] Mission 11: 비즈니스 로직 수정
- [x] Mission 12: Plotly 대시보드 생성
- [x] Mission 13: 정제 데이터 저장

---

## 💡 학습 정리

### 데이터 정제 핵심 프로세스

| 단계 | 작업 | 핵심 메소드 | 비고 |
|------|------|------------|------|
| 1. 진단 | 결측치/이상치/타입오류 파악 | info(), isnull(), describe() | 전체 현황 파악 |
| 2. 타입 변환 | 날짜/문자열/숫자 포맷 통일 | pd.to_datetime(), str.strip() | 분석 가능한 형식으로 |
| 3. 이상치 처리 | 극단값 수정 또는 대체 | loc[], transform() | 삭제보다 수정 우선 |
| 4. 결측치 대체 | 최빈값/평균/중앙값 대체 | fillna(), mode(), groupby | 비즈니스 로직 고려 |
| 5. 중복 제거 | 완전 중복 삭제 | drop_duplicates() | 첫 번째만 유지 |
| 6. 검증 | 비즈니스 규칙 재확인 | 조건 필터링 | 규칙 위반 수정 |
| 7. 저장 | 정제 데이터 파일 출력 | to_csv() | UTF-8 인코딩 |

### 💡 실무 팁

1. **원본 보존**: `df_clean = df.copy()`로 작업
2. **단계별 검증**: 각 정제 작업 후 `df.info()`, `df.head()` 확인
3. **비즈니스 우선**: 도메인 지식 활용한 합리적 대체
4. **시각화 필수**: Plotly로 정제 전후 비교
5. **문서화 습관**: 정제 규칙을 주석으로 기록
6. **자동화 고려**: 반복 작업은 함수로 만들기
7. **품질 리포트**: 최종 결과를 숫자로 요약

### 🎯 핵심 코드 패턴

```python
# 1. 결측치 대체
df['column'].fillna(df['column'].mode()[0], inplace=True)

# 2. 제품별 평균으로 대체
df['price'] = df.groupby('product')['price'].transform(
    lambda x: x.fillna(x.mean())
)

# 3. 이상치 수정
abnormal_mask = (df['price'] < 100) | (df['price'] > 1000000)
df.loc[abnormal_mask, 'price'] = product_avg_price

# 4. 비즈니스 로직
df.loc[df['status'] == '취소', 'amount'] = 0
```

**데이터 정제는 지루하지만 가장 중요한 작업입니다!**

**실무에서는 데이터 분석 시간의 70%가 정제입니다!**